# Notebook for CPEN 534 Course projects

This notebook contains the scripts to run all experiments and generate all figures in the poster (and its Extended Abstract).
Please follow the these step:
1. Run `pipenv install` to install the python dependencies specified in `Pipfile` and `Pipfile.lock`.
2. Ensure `go version` is `1.22.1`.
3. Update the values of `path_to_graph`, `path_to_project`, and `threads` in the next cell.
4. Run all cells in this notebook with the python environment created in Step 1.
5. The results will be saved in the directory `results/2024-cpen534/`.

In [126]:
# 
project_path = "/home/juntong/lollipop-partitioning" # root of the repository
path_hive_comments = "/home/juntong/hive-comments.txt"
path_wikipedia_growth = "/home/juntong/wikipedia-growth.txt"
path_eth_transfers = "/home/juntong/eth-transfers.txt"
path_flickr = "/home/juntong/flickr-growth.txt"
# path_roadnet = "/home/juntong/roadNet-CA.txt.shuffled" # shuffled with random edge weights added # ignore this for now

default_thread = 32

In [127]:
import subprocess as sp
from pathlib import Path
import shutil
import re
import time
import statistics
import os, signal
from dataclasses import dataclass
import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates

mpl.rcParams['figure.dpi'] = 600

In [128]:
go = Path(shutil.which("go"))
project_path = Path(project_path).expanduser()
path_hive_comments = Path(path_hive_comments).expanduser()
path_wikipedia_growth = Path(path_wikipedia_growth).expanduser()
path_eth_transfers = Path(path_eth_transfers).expanduser()
path_flickr = Path(path_flickr).expanduser()
# path_roadnet = Path(path_roadnet).expanduser()
assert go.exists()
assert project_path.exists()
assert path_hive_comments.exists()
assert path_wikipedia_growth.exists()
assert path_eth_transfers.exists()
assert path_flickr.exists()
# assert path_roadnet.exists()

# TODO: check hashes of graph files

In [129]:
def create_results_directory(top_dir: Path):
    results_dir = top_dir / f"2024-cpen534"
    log_dir = results_dir / "log"
    ts_dir = results_dir / "ts"
    output_dir = results_dir / "output"
    results_dir.mkdir(exist_ok=True)
    log_dir.mkdir(exist_ok=True)
    ts_dir.mkdir(exist_ok=True)
    output_dir.mkdir(exist_ok=True)
    return results_dir, output_dir, log_dir, ts_dir

(project_path / "results").mkdir(exist_ok=True)
results_dir, output_dir, log_dir, ts_dir = create_results_directory(project_path / "results")
path_timeseries_output = project_path / "results" / "push-relabel-timeseries.csv"
path_timeseries_flow_output = project_path / "results" / "push-relabel-timeseries-flow.csv"
path_partitioning_output = project_path / "results" / "partitioning-stats.csv"

In [130]:
@dataclass(kw_only=True)
class Graph:
    name: str
    path: Path
    pw: int
    pt: int
    undirected: bool = False

@dataclass(kw_only=True)
class ExperimentConfig:
    graph: Graph
    cmd: str
    threads: int = default_thread
    dynamic: bool = False
    target_ingest_rate: int = 0
    mla: bool = False
    mlaLoad: str = "v"
    mlaAlpha: float = 0.0
    mlaBatch: int = 1

@dataclass(kw_only=True)
class BasicTestCase(ExperimentConfig):
    threads: int = default_thread

graph_hive_comments = Graph(name="Hive-comments", path=path_hive_comments, pw=None, pt=1)
graph_wikipedia_growth = Graph(name="Wikipedia-growth", path=path_wikipedia_growth, pw=None, pt=2)
graph_eth_transfers = Graph(name="Eth-transfers", path=path_eth_transfers, pw=1, pt=2)
graph_flickr_growth = Graph(name="Flickr-growth", path=path_flickr, pw=None, pt=2)
# graph_roadnet = Graph(name="RoadNet-CA", path=path_roadnet, pw=1, pt=None)

In [131]:
def plot_histogram(g: Graph, xlog: bool = False):
    degrees = dict()
    with open(g.path) as file:
        for line in file:
            src, _ = line.split(maxsplit=1)
            if src not in degrees:
                degrees[src] = 1
            else:
                degrees[src] += 1

    fig, ax = plt.subplots(1, 1, figsize=(2, 2))
    ax.set(title=f'{g.name}', xlabel='Out Degree', ylabel='# of Vertices', yscale='log', xlim=[0, max(degrees.values())])
    if xlog:
        ax.set(xscale='log')
    ax.hist(degrees.values(), bins=100)

    fig.tight_layout(pad=0)
    fig.savefig(output_dir / f"degree-histogram-{g.name}.pdf", bbox_inches='tight')

# plot_histogram(graph_wikipedia_growth)
# plot_histogram(graph_hive_comments, xlog=False)
# plot_histogram(graph_flickr_growth)

In [132]:
@dataclass
class Result:
    cfg: ExperimentConfig
    # time_alg: int
    # time_general: int
    edge_cuts: int
    num_edges: int
    df_partitioning: pd.DataFrame
    balance_v: float
    balance_e: float
    balance_msg: float
    edge_cut: float
    inter_partition_msg: float

def get_result(cfg: ExperimentConfig, path_log: Path, partitioning_output: Path) -> Result:
    log = path_log.read_text()

    # time_alg = re.findall(r"Termination: ([0-9]+)", log)
    # assert len(time_alg) == 1 
    # time_alg = int(time_alg[0])

    # time_general = re.findall(r"Total including streaming: ([0-9]+)", log)
    # assert len(time_general) == 1 
    # time_general = int(time_general[0])

    edge_cuts = re.findall(r"Edge Cuts: ([0-9]+)", log)
    assert len(edge_cuts) == 1 
    edge_cuts = int(edge_cuts[0])

    num_edges = re.findall(r"Edges: +([0-9]+)", log)
    assert len(num_edges) == 1 
    num_edges = int(num_edges[0])

    df_partitioning = pd.read_csv(partitioning_output)

    assert df_partitioning.loc[:, "out_edges"].sum() == num_edges
    assert df_partitioning.loc[:, "in_edges"].sum() == num_edges

    vertices = df_partitioning.loc[:, "vertices"].values
    edges = df_partitioning.loc[:, "out_edges"].values
    msgs_all = (df_partitioning.loc[:, "msg_recv_local"] + df_partitioning.loc[:, "msg_recv_remote"]).values
    msgs_remote = df_partitioning.loc[:, "msg_recv_remote"].values
    balance_v = max(vertices) / (sum(vertices) / len(vertices))
    balance_e = max(edges) / (sum(edges) / len(edges))
    balance_msg = max(msgs_all) / (sum(msgs_all) / len(msgs_all))
    edge_cut = edge_cuts / num_edges
    inter_partition_msg = sum(msgs_remote) / sum(msgs_all)

    return Result(cfg=cfg, edge_cuts=edge_cuts, num_edges=num_edges, df_partitioning=df_partitioning, balance_v=balance_v, balance_e=balance_e, balance_msg=balance_msg, edge_cut=edge_cut, inter_partition_msg=inter_partition_msg)

In [133]:
def run_algorithm(cfg: ExperimentConfig, repeat_index: int = 0, correctness: bool = True, add_flags: list[str] = [], suffix: str = "", no_skip: bool = False) -> Path:
    # compile command and log filename
    cmd = [go, "run", project_path / "cmd" / f"lp-{cfg.cmd}", "-nc", f"-t={cfg.threads}", f"-g={cfg.graph.path}", "-tg=1"]
    log_filename = f"{cfg.cmd}-t={cfg.threads}-g={cfg.graph.path.name}"
    if cfg.cmd == "sssp":
        cmd.append("-i=38806")
        log_filename += "-i=38806"
    if cfg.graph.undirected:
        cmd.append("-u")
        log_filename += "-u"
    if cfg.graph.pw:
        cmd += [f"-pw={cfg.graph.pw}"]
        log_filename += f"-pw={cfg.graph.pw}"
    if cfg.graph.pt:
        cmd += [f"-pt={cfg.graph.pt}"]
        log_filename += f"-pt={cfg.graph.pt}"
    if cfg.dynamic:
        cmd += [f"-de=100000"]
        log_filename += f"-de=100000"
    if cfg.target_ingest_rate != 0:
        cmd += [f"-dr={cfg.target_ingest_rate}"]
        log_filename += f"-dr={cfg.target_ingest_rate}"
    if cfg.mla:
        cmd += [f"-mla", f"-ml={cfg.mlaLoad}", f"-ma={cfg.mlaAlpha}", f"-mb={cfg.mlaBatch}"]
        log_filename += f"-mla-ml={cfg.mlaLoad}-ma={cfg.mlaAlpha}-mb={cfg.mlaBatch}"
    

    if correctness:
        cmd += ["-c"]
        log_filename += "-c"

    cmd += add_flags
    log_filename += "".join(add_flags)

    log_filename += f"{suffix}.{repeat_index}.log"

    # Run command
    log_path = log_dir / log_filename
    print(f"Command: {cmd}")
    print(f"Log path: {log_path}")

    if (not no_skip) and log_path.exists():
        print(f"Skipped as the log file exists")
        return log_path

    path_log_temp = log_dir / f"_running-{log_filename}"
    with open(path_log_temp, "w+t") as log_file:
        def preexec_fn():
            os.setpgrp()
        process = sp.Popen(cmd, cwd=project_path, stdout=log_file, stderr=sp.STDOUT, preexec_fn=preexec_fn)
        time.sleep(0.5)
        try:
            returncode = process.wait()
        except KeyboardInterrupt as ki:
            os.killpg(os.getpgid(process.pid), signal.SIGTERM)
            process.kill()
            raise ki
        if returncode != 0:
            assert False, f"Return code is none-zero: {returncode}"
    
    path_log_temp.rename(log_path)
    print(f"Log saved")
    
    return log_path

def run_algorithm_partitioning(cfg: ExperimentConfig, **kwargs):
    path_log = run_algorithm(cfg=cfg, **kwargs)

    path_partitioning = ts_dir / (path_log.name[:-4] + f".csv")
    if path_partitioning.exists():
        print(f"Partitioning output already exists: {path_partitioning}")
    else:
        assert path_partitioning_output.exists(), "No partitioning output found"
        path_partitioning_output.rename(path_partitioning)
        print(f"Partitioning output saved to: {path_partitioning}")

    return get_result(cfg=cfg, path_log=path_log, partitioning_output=path_partitioning)

In [ ]:
class HashExperiment:
    def __init__(self):
        self.experiments = [
            ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=False),
            ExperimentConfig(graph=graph_flickr_growth, cmd="sssp", mla=False),
            ExperimentConfig(graph=graph_wikipedia_growth, cmd="sssp", mla=False)
        ]

    def run(self):
        self.results = []
        for experiment in self.experiments:
            result = run_algorithm_partitioning(experiment)
            self.results.append([result.cfg.graph.name, result.balance_v, result.balance_e, result.balance_msg, result.edge_cut])
        self.df_results = pd.DataFrame(self.results, columns=["graph", "|V|", "|E|", "|Msgs|", "% of Edge Cut"])
        print(self.df_results)

    def plot_balance(self, figsize):
        df_balance = pd.DataFrame(self.df_results.loc[:, ["|V|", "|E|", "|Msgs|"]].T)
        df_balance.columns = self.df_results.loc[:, "graph"].values

        fig, ax = plt.subplots(1, 1, figsize=figsize)
        df_balance.plot(ax=ax, kind="bar", stacked=False, legend=False, edgecolor="black")
        hatches = ['/', '/', '/', 'o', 'o', 'o', 'x', 'x', 'x']
        for bar, ha in zip(ax.patches, hatches):
            bar.set_hatch(ha)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
        ax.legend(loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.3))
        ax.grid(axis='y')
        ax.set(ylim=(0, 3), axisbelow=True, ylabel="Load Imbalance")
        
        fig.tight_layout(pad=0.5)
        return fig

    def plot_edge_cut(self, figsize):
        df_edgecut = pd.DataFrame(self.df_results.loc[:, ["% of Edge Cut"]].T)
        df_edgecut.columns = self.df_results.loc[:, "graph"].values

        fig, ax = plt.subplots(1, 1, figsize=figsize)
        df_edgecut.plot(ax=ax, kind="bar", legend=False, edgecolor = "black")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
        ax.grid(axis='y')
        ax.set(ylim=(0, 1), axisbelow=True)
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

        hatches = ['/', 'o', 'x']
        for bar, ha in zip(ax.patches, hatches):
            bar.set_hatch(ha)
        
        fig.tight_layout(pad=0.5)
        return fig

hash_experiment = HashExperiment()
hash_experiment.run()
height = 3
fig_balance = hash_experiment.plot_balance((5.5,height))
fig_edgecut = hash_experiment.plot_edge_cut((2,height))
fig_balance.savefig(output_dir / "hash-balance.pdf", bbox_inches='tight')
fig_edgecut.savefig(output_dir / "hash-edgecut.pdf", bbox_inches='tight')

In [ ]:
class LoadCriteriaExperiment:
    def __init__(self):
        self.experiments = [
            ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=False),
            ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=True, mlaLoad="v"),
            ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=True, mlaLoad="e"),
            ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=True, dynamic=True, mlaLoad="msg"),
        ]
        self.names = ["Hash", "Balancing |V|", "Balancing |E|", "Balancing #Msgs+0.01|E|"]

    def run(self):
        self.results = []
        for name, experiment in zip(self.names, self.experiments):
            result = run_algorithm_partitioning(experiment)
            self.results.append([name, result.balance_v, result.balance_e, result.balance_msg])
        self.df_results = pd.DataFrame(self.results, columns=["Balancing Criteria", "|V| Balance", "|E| Balance", "#Msgs Balance"])
        self.df_results.set_index("Balancing Criteria", inplace=True)

    def plot(self, figsize):
        def plot_optimize_criteria(ax, df_full, balance_criteria):
            df = pd.DataFrame(df_full.loc[balance_criteria, :]).T
            df.plot(ax=ax, kind="bar", stacked=False, legend=False, edgecolor="black")
            ax.get_xaxis().set_visible(False)
            ax.grid(axis='y')
            ax.set(ylim=(0, 3), axisbelow=True, title=balance_criteria)
            hatches = ['/', 'o', 'x']
            for bar, ha in zip(ax.patches, hatches):
                bar.set_hatch(ha)
            handles, labels = ax.get_legend_handles_labels()
            fig.legend(handles, labels, loc='lower center', ncols=len(handles), bbox_to_anchor=(0.5, -0.05))

        fig, axs = plt.subplots(2, 2, figsize=figsize)
        plot_optimize_criteria(axs[0, 0], self.df_results, self.names[0])
        plot_optimize_criteria(axs[0, 1], self.df_results, self.names[1])
        plot_optimize_criteria(axs[1, 0], self.df_results, self.names[2])
        plot_optimize_criteria(axs[1, 1], self.df_results, self.names[3])
        axs[0, 0].set(ylabel="Load Imbalance")
        axs[1, 0].set(ylabel="Load Imbalance")

        fig.tight_layout(pad=0.5)
        return fig

load_criteria = LoadCriteriaExperiment()
load_criteria.run()
fig = load_criteria.plot(figsize=(8,5))
fig.savefig(output_dir / "load-criteria.pdf", bbox_inches='tight')

In [ ]:
class AlphaExperiment:
    def __init__(self, alphas):
        self.experiments = []
        for alpha in alphas:
            self.experiments.append(ExperimentConfig(graph=graph_hive_comments, cmd="sssp", mla=True, mlaAlpha=alpha, mlaLoad="e"))
            self.experiments.append(ExperimentConfig(graph=graph_flickr_growth, cmd="sssp", mla=True, mlaAlpha=alpha, mlaLoad="v"))
            self.experiments.append(ExperimentConfig(graph=graph_wikipedia_growth, cmd="sssp", mla=True, mlaAlpha=alpha, mlaLoad="v"))

    def run(self):
        self.results = []
        for experiment in self.experiments:
            result = run_algorithm_partitioning(experiment)
            self.results.append([experiment.graph.name, experiment.mlaLoad, experiment.mlaAlpha, result.edge_cut, result.inter_partition_msg, result.balance_v, result.balance_e, result.balance_msg])
        self.df_results = pd.DataFrame(self.results, columns=["Graph", "Load", "Alpha", "Edge-Cut", "Inter-Partition Msg", "|V| Balance", "|E| Balance", "#Msgs Balance"])

    def plot(self, figsize):
        load_criteria_name = {"v": "|V|", "e": "|E|", "msg": "#Msg"}
        def plot_ax(ax, graph, load_criteria, xlim, legend):
            df = self.df_results[self.df_results["Graph"] == graph]
            x = df["Edge-Cut"]*100
            ax.plot(x, df["|V| Balance"], '-o', label="|V| Balance")
            ax.plot(x, df["|E| Balance"], '-x', label="|E| Balance")
            ax.plot(x, df["#Msgs Balance"], '-v', label="#Msgs Balance")
            ax.grid()
            ax.set(xlim=(xlim, 100), ylim=(0, 13), axisbelow=True, xlabel="Edge-Cut (%)", title=f"{graph}")
            if legend:
                ax.legend()

        fig, axs = plt.subplots(1, 3, figsize=figsize, sharey=True)
        plot_ax(axs[0], "Hive-comments", "e", 80, False)
        plot_ax(axs[1], "Flickr-growth", "v", 65, False)
        plot_ax(axs[2], "Wikipedia-growth", "v", 80, True)
        axs[0].set(ylabel="Load Imbalance")
        fig.tight_layout(pad=0.5)
        return fig


alpha_experiment = AlphaExperiment([0, 1/8, 2/8, 4/8, 1, 2, 4, 8, 12])
# alpha_experiment = AlphaExperiment([0, 1, 8, 64, 128])
alpha_experiment.run()
fig = alpha_experiment.plot((8,3))
fig.savefig(output_dir / "alpha.pdf", bbox_inches='tight')

In [ ]:
class BatchExperiment:
    def __init__(self, alphas, batches):
        self.experiments = []
        self.batches = batches
        self.line_styles = ["-o", "-x", "-v", "-s", "-*"]
        assert len(self.line_styles) >= len(self.batches)
        for graph in [graph_hive_comments, graph_flickr_growth, graph_wikipedia_growth]:
            for b in batches:
                for alpha in alphas:
                    self.experiments.append(ExperimentConfig(graph=graph, cmd="sssp", mla=True, mlaAlpha=alpha, mlaBatch=int(b)))

    def run(self):
        self.results = []
        for experiment in self.experiments:
            result = run_algorithm_partitioning(experiment)
            self.results.append([experiment.graph.name, experiment.mlaLoad, experiment.mlaAlpha, experiment.mlaBatch, result.edge_cut, result.inter_partition_msg, result.balance_v, result.balance_e, result.balance_msg])
        self.df_results = pd.DataFrame(self.results, columns=["Graph", "Load", "Alpha", "Batch", "Edge-Cut", "Inter-Partition Msg", "|V| Balance", "|E| Balance", "#Msgs Balance"])

    def plot(self, figsize):
        def plot_line(df, ax, line_style, batch_size):
            batch_size_name = {1: "$1$", 1e3: "$10^3$", 1e4: "$10^4$", 1e5: "$10^5$", 1e6: "$10^6$"}[batch_size]
            df = df[df["Batch"] == batch_size]
            x = df["Edge-Cut"]*100
            ax.plot(x, df["|V| Balance"], line_style, label=f"Batch of {batch_size_name}")
        def plot_graph(df, graph, ax, xlim, ylim):
            df = df[df["Graph"] == graph]
            for batch, style in zip(self.batches, self.line_styles):
                plot_line(df, ax, style, batch)
            ax.grid()
            ax.set(xlim=(xlim, 100), ylim=(0, ylim), axisbelow=True, xlabel="Edge-Cut (%)", ylabel="Load Imbalance", title=graph)

        df = self.df_results
        fig, axs = plt.subplots(1, 3, figsize=figsize)
        plot_graph(df, "Hive-comments", axs[0], 15, 17.5)
        plot_graph(df, "Flickr-growth", axs[1], 20, 9)
        plot_graph(df, "Wikipedia-growth", axs[2], 40, 15)
        handles, labels = axs[2].get_legend_handles_labels()
        fig.legend(handles, labels, loc='lower center', ncols=len(handles), bbox_to_anchor=(0.5, -0.11))
        fig.tight_layout(pad=0.5)
        return fig
        

batch_experiment = BatchExperiment([0, 1/8, 2/8, 4/8, 1, 2, 4, 8], [1, 1e3, 1e4, 1e5, 1e6])
batch_experiment.run()
fig = batch_experiment.plot((9*0.9,3*0.9))
fig.savefig(output_dir / "batch.pdf", bbox_inches='tight')